<a href="https://colab.research.google.com/github/aidash8/optimizing_returning_the_names/blob/main/Optimizing_the_returning_the_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing "returning_the_names" model (gpt-4o)
This colab is trying various LLM techniques to optimize the production model (gpt-4o + prompt engineering + RAG through trie) that was used in the October 29 2024 commemoration.

The techniques used for optimization are:
- RAG with embeddings
  - creating embeddings for all Russian abbreviations dictionary (done)
  - creating embeddings for all Russian Geographical names
- fine tuning the model through data from prior years (done)
- make the model to think from this [paper](https://arxiv.org/pdf/2410.10630)

## Read data and set up the working directory and libraries

In [2]:
# Import the important libraries
import os
import sqlite3
import pandas as pd
import json
import asyncio
import re
import time
import openai
from scipy import spatial  # for calculating vector similarities for search

In [4]:
# Mount the google drive
from google.colab import drive
import os

# Check if the directory is empty or not
if os.path.exists('/content/drive') and os.listdir('/content/drive'):
  # If it's not empty, remove the directory
  !rm -rf /content/drive

# Now mount the drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Create a working directory
path = "/content/drive/MyDrive/Projects/optimizing_returning_the_names"
os.makedirs(path, exist_ok=True)

# Change the current directory to the working directory
%cd $path

/content/drive/MyDrive/Projects/optimizing_returning_the_names


#### Download the production model repo.

- The repo has the USSR repressions data as sqlite
    - The data is a snapshot of a bigger database. The snapshot has only needed columns and only people who were shot during the purges

In [ ]:
# In order to clone, you need a github's personal access token (https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens)
# Put the personal access token in the working directory in the file named "github_access_key.txt"
# or just copy past to the variable personal_access_token

personal_access_token = 'Your personal access key'
with open('github_access_key.txt', 'r') as f:
    personal_access_token = f.read().strip()

!git clone https://$personal_access_token@github.com/compositor/returning_the_names.git

In [ ]:
# Install all required libraries
!pip install -r returning_the_names/requirements.txt
!pip install transformers
!pip install tiktoken

#### Read and clean data
Data can be cleaned using functions from the returning_the_names repo, but Colab was giving me an error about locale (doesn't have the right locale installed). Couldn't fix it, so decided to copy the functions here and modify them to for the dataframe use

In [4]:
COLUMN_NAMES = [
    "id",
    "name",
    "birth_date",
    "gender",
    "place_of_birth",
    "occupation",
    "place_of_residence",
    "execution_place",
    "execution_date",
]

conn = sqlite3.connect('returning_the_names/data/october.sqlite3')
cur = conn.cursor()
cur.execute("SELECT * FROM october")
results = cur.fetchall()

df = pd.DataFrame(results)
df.columns = COLUMN_NAMES
print(len(df))
df.head(2)

143432


,id,name,birth_date,gender,place_of_birth,occupation,place_of_residence,execution_place,execution_date
0,888372,Назаров Алексей Матвеевич,1897,мужчина,Томская губ.,Рабочий зверосовхоза,"Алтайский кр., Бийский р-н",г. Бийске,18.11.1937
1,490842,Назаров Емельян Васильевич,1888,мужчина,Томская губ.,"Член колхоза ""Память Ленина""","Грязнухинский р-н, с. Кокши",г. Бийске,28.03.1938


In [5]:
from typing import Optional, Tuple, Any
from datetime import datetime, date
import locale
from dateutil.relativedelta import relativedelta
import re

def clean_date(x: str) -> date:
    if not x:
        return None
    x_orig = x.strip()
    if x == "1906 (по др. данным, в 1905)":
      x = '01.01.1906'
    elif x == "28.03 (9.04).1891":
      x = "28.03.1891"
    elif x == "31.09.1938":
      x = "30.09.1938"
    elif x == '01(13). 02.1884':
      x = "01.02.1884"
    elif x == '1898 (по данным протокола №82- 1905)':
      x = "01.01.1898"
    elif x == '11.03 (28.02).1888':
      x = "11.03.1888"
    elif x == '11.10.08.1933':
      x = "11.08.1933"
    elif x == '28.02(?).1938':
      x = "28.02.1938"
    x = re.sub(r'\(\d+\)', '', x)
    x = x.split(";")[-1]
    x = x.split(".")
    if len(x) == 1:
        x = ["01", "01", x[0].strip()]
    elif len(x) == 2:
        x = ["01", x[0].strip(), x[1].strip()]
    try:
      return datetime.strptime('.'.join(x), "%d.%m.%Y").date()
    except:
      return ""


def convert_to_rus_datetime(x: Optional[date]) -> str:
    if x is None:
        return ""
    locale.setlocale(locale.LC_TIME, "ru_RU.UTF-8")
    return x.strftime("%d %B %Y")


def create_age(date_of_demise: date, dob: date) -> str:
    if date_of_demise is None or dob is None:
        return ""
    if date_of_demise == "" or dob == "":
        return ""
    years = relativedelta(date_of_demise, dob).years
    if years < 10:
      print(date_of_demise, dob)
        # raise Exception(f"Age is too low. Likely incorrect: {date_of_demise, dob}")
    return str(years)


def clean_profession(x: Optional[str]) -> str:
    if not x:
        return x
    x = re.sub("по месту рождения,", "", x)
    return x


def normalize_name(name: str) -> str:
    li = name.split(" ")
    li2 = li[1:]
    li2.extend(li[:1])
    return " ".join(li2)

def create_age_final(x) -> str:
    dob = clean_date(x["birth_date"])
    if not x["execution_date"]:
        raise Exception("Date of death is unknown")

    date_of_demise = clean_date(x["execution_date"])

    return create_age(date_of_demise, dob) if date_of_demise and dob else ""

def manual_cleaning(x):
    if x['id'] == 418499:
        x["birth_date"] = '20.04.1888' # openlist.wiki (info from Книга памяти "Корейцы - жертвы политических репрессий в СССР")
    elif x['id'] == 517801:
        x['execution_date'] = '02.08.1930' # https://drevo-info.ru/articles/26121.html
    elif x['id'] == 1068479:
        x['execution_date'] = '14.10.1938' # https://lists.memo.ru/d1/f464.htm
    return x

manual_cleaning_ids = [418499, 517801, 1068479]
df.loc[df['id'].isin(manual_cleaning_ids)] = df.loc[df['id'].isin(manual_cleaning_ids),].apply(lambda x: manual_cleaning(x), axis=1)
df['name'] = df['name'].apply(normalize_name)
df['age'] = df.apply(lambda x: create_age_final(x), axis=1)
df.head(2)

1938-01-22 1937-01-01
1920-01-01 1912-01-01
1938-08-05 1936-01-01
1938-04-08 1938-01-01


,id,name,birth_date,gender,place_of_birth,occupation,place_of_residence,execution_place,execution_date,age
0,888372,Алексей Матвеевич Назаров,1897,мужчина,Томская губ.,Рабочий зверосовхоза,"Алтайский кр., Бийский р-н",г. Бийске,18.11.1937,40
1,490842,Емельян Васильевич Назаров,1888,мужчина,Томская губ.,"Член колхоза ""Память Ленина""","Грязнухинский р-н, с. Кокши",г. Бийске,28.03.1938,50


## The production model

The production model is in the returning_the_names repo.

We used gpt-4o (the full model) which was tuned by
- prompt engineering
- RAG through tries: retrieval of the new information through exact matches. The dictionary of abbreviations was stored in a trie structure where the keys are normalized abbreviations and its meaning and the values are abbreviations and its meaning.
  - Normalization is done by leaving only letters, no special charactes, spaces, numbers and etc

In [6]:
# Get into the returning_the_names folder in order to run the production model
%cd returning_the_names

/content/drive/MyDrive/Projects/optimizing_returning_the_names/returning_the_names


In [10]:
# Put your OPEN AI Key to returning_the_names folder in the secret.txt file. Here is the tutorial to the OPEN AI API: https://platform.openai.com/docs/quickstart
OPENAI_KEY = 'Your OPEN AI Key'
!echo $OPENAI_KEY > secret.txt

[Errno 2] No such file or directory: 'returning_the_names'
/content/drive/MyDrive/Projects/optimizing_returning_the_names/returning_the_names


In [7]:
# Read your OPEN AI Key if it's not saved in OPENAI_KEY
with open('secret.txt', 'r') as f:
    OPENAI_KEY = f.read().strip()

In [8]:
from openai import AsyncOpenAI
from src.ai_client import AiClient

full = True # the full model aka gpt-4o
input_to_client = AiClient(full) # the structure that has info about the prompt and model

# async OPEN AI client for faster communication
client = AsyncOpenAI(api_key=OPENAI_KEY, timeout=60, max_retries=3)

In [23]:
# Pick a random example to see how the current model works
example = df.sample()
example

,id,name,birth_date,gender,place_of_birth,occupation,place_of_residence,execution_place,execution_date,age
19041,1122501,Владимир Григорьевич Вейнштейн,1904,мужчина,"Венгрия, г. Комарово",НИИ-20: Слесарь-механик,"Москва, ул. Горького, д. 110, кв. 3","Московская обл., Бутово",10.06.1938,34


In [24]:
from src.cards import RawCard
from src import ai_prompts

KEEP_COLS = ['id', 'name', 'age', 'gender', 'place_of_birth', 'place_of_residence', 'occupation', 'execution_date', 'execution_place']

record = RawCard(**example[KEEP_COLS].to_dict(orient='records')[0])
card = record.pretty_print_without_id()
print(card)


Имя Отчество Фамилия: Владимир Григорьевич Вейнштейн  
Возраст: 34
Пол: мужчина
Место рождения: Венгрия, г. Комарово
Род занятий: НИИ-20: Слесарь-механик
Дата расстрела: 10.06.1938
Место расстрела: Московская обл., Бутово      



The production model's messages to send to OPEN AI consists of:
- system message 1: this is the prompt.
  - We asked LLM to be a competent editor that creates cards for people to read loud in an event.
  - We asked to expand the abbreviations
  - We asked to expand correclty geographical objects from the USSR time
  - We asked to correct grammatical errors
  - We asked to return the card in the following format:
    - first row: Name
    - second row: Age
    - third row: the place where the person was born
    - forth row: short biography of the person (occupation and where lived if the place of birth is different from the place of the birth)
    - fifth row: information about the death
- system message 2: we added information from the Russian abbreviation dictionary here. We extracted the abbreviations that might be useful through RAG
- assistent message: examples of the good cards
- user message: input information from which we want to create the card for reading during the commemoration

In [30]:
# Get the prompt for OPEN AI (messages) for this particular example
# You can see that in the 2nd system message there is explanation of "Ушосдор", "УНКВД"
messages = input_to_client.get_messages(record)
messages

[{'role': 'system',
  'content': [{'type': 'text',
    'text': '\nТы - грамотный редактор, феминист. \nТвоя задача - подготовить для зачитывания карточку о расстрелянном человеке.\nКарточка должна звучать естественно и красиво. Нельзя использовать бюрократический и канцелярский язык.\n\nВерни текст, в котором раскрыты сокращения, и согласованы все падежи и формы слов.\nНе раскрывай самые распространенные сокращения, такие как Советская Социалистическая Республика, СССР.\nВсегда используй справочник https://abbr_rus.academic.ru/ . Если возможно несколько вариантов, оставь сокращение как есть. \nПодумай хорошо над сокращением.\nПри раскрытии сокращений перед населенными пунктами используй знание о настоящих географических объектах времен Советского Союза.\nPlease pay extra attention on expanding abbreviations. For example, пуст. should be пустоши, but not поселке or пустыне.\nИспользуй названия настоящих населенных пунктов. Если надо, исправь написание на правильное, везде, кроме имен.\n

In [31]:
# We used temperature=0 to eliminate hallucinations

print(f"The model is: {input_to_client.model}")
completion = await client.chat.completions.create(
            model=input_to_client.model, messages=messages, temperature=0
        )
result = completion.choices[0].message.content
print("The production model result:\n")
print(result)
print()

The model is: gpt-4o
The production model result:

Вейнштейн Владимир Григорьевич  
34 года  
Родился в городе Комарово в Венгрии  
Слесарь-механик в Научно-исследовательском институте-20  
Расстрелян 10 июня 1938 года в Бутово Московской области  



## Optimization of the production model

Since the gpt-4o-mini is a worse model than gpt-4o, we optimize the gpt-4o-mini in order to see the improvements. This will help us to understand the direction we need to go to improve further the gpt-4o model for this task.

The reason why we optimize the mini-model is that we collected all good examples where the model didn't work for mini-model. We don't have good examples where the full model doesn't work.

We assume that all steps we do that improves the mini model will improve better model (the full model)


#### Examples of the gpt-4o-mini defficiencies

The result of the mini model is not bad. Only thing the mini model didn't expand "Норильлага".

Let's check examples where the mini model fails to compare to the full model

In [ ]:
# Example of the mini model that hallucinates (wrongly expands the abbreviations): Зав. сектором горЗО
# the full model gives the right answer (right expantion of 'горЗО') with and without RAG

id = 1222123 # горЗО
record = df.loc[df.id == id, KEEP_COLS].to_dict(orient='records')[0]
record = RawCard(**record)
card = record.pretty_print_without_id()
print(card)
print()
print(f"The prod model stems: {record.get_stems()}")
print()
custom = ai_prompts.get_sub(record.get_stems())
print(f"The length of the additional prompt created through RAG: {len(custom.split(chr(10)))}")



Имя Отчество Фамилия: Венедикт Иванович Катков  
Возраст: 31
Пол: мужчина
Место рождения: None
Род занятий: Зав. сектором горЗО
Дата расстрела: 01.07.1938
Место расстрела: г. Барнаул      


The prod model stems: {'зав', 'барна', 'гор', 'сектор'}

The length of the additional prompt created through RAG: 61


In [ ]:
# the production model
messages = input_to_client.get_messages(record)
completion = await client.chat.completions.create(
            model=input_to_client.model, messages=messages, temperature=0
        )
result = completion.choices[0].message.content
print("The production model:\n")
print(result)
print()
print(f"Total length of additional prompt: {len(messages[1]['content'][0]['text'])}")
print(f"Number of abbr in the additional prompt: {len(messages[1]['content'][0]['text'].split(chr(10)))}")

# the production model without RAG. It's shown why we need RAG
messages_wo_abbr = [m for i, m in enumerate(messages) if i != 1]
completion = await client.chat.completions.create(
            model=input_to_client.model, messages=messages_wo_abbr, temperature=0
        )
result = completion.choices[0].message.content
print()
print(f"The {input_to_client.model} without RAG\n")
print(result)

# check the second attempt for the production model without RAG (sometimes it gives the correct answer)
completion = await client.chat.completions.create(
            model=input_to_client.model, messages=messages_wo_abbr, temperature=0
        )
result = completion.choices[0].message.content
print()
print(f"The {input_to_client.model} without RAG, 2nd attempt\n")
print(result)

The production model:

Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского земельного отдела  
Расстрелян 1 июля 1938 года в Барнауле  

Total length of additional prompt: 3564
Number of abbr in the additional prompt: 65

The gpt-4o without RAG

Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского отдела здравоохранения  
Расстрелян 1 июля 1938 года в Барнауле  

The gpt-4o without RAG, 2nd attempt

Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского отдела здравоохранения  
Расстрелян 1 июля 1938 года в Барнауле  


You can see the production model works well with the knowdledge extracted through RAG. It did't expand correctly "горЗО" without RAG. It expanded it as "отдела здравоохранения" while it needs to be "городского земельного отдела"

In [ ]:
# mini model hallucinates even with RAG
# it takes the first abbreviations from the RAG list
model = "gpt-4o-mini"
temperature = 0

# with RAG
messages = input_to_client.get_messages(record)
completion = await client.chat.completions.create(
            model=model, messages=messages, temperature=temperature
        )
result = completion.choices[0].message.content
print(f"{model} model with RAG:\n")
print(result)

gpt-4o-mini model with RAG:

Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского отдела  
Расстрелян 1 июля 1938 года в Барнауле  


In [ ]:
# but stopes the hallucination if the correct abbreviation is on the top of the RAG list
messages[1]['content'][0]['text'] = re.sub('"горзо": "городской земельный отдел"','',messages[1]['content'][0]['text'])
messages[1]['content'][0]['text'] = '\nИспользуй этот справочник для расшифровки аббревиатур\n\n"горзо": "городской земельный отдел"' + messages[1]['content'][0]['text'][59:]

print(messages[1]['content'][0]['text'][:200])
completion = await client.chat.completions.create(
            model=model, messages=messages, temperature=temperature
        )
result = completion.choices[0].message.content
print("---------------------")
print(result)


Используй этот справочник для расшифровки аббревиатур

"горзо": "городской земельный отдел" "завод (карт.), заимка (карт.), запад, западный, -ая, -ое (карт.), застава (карт.)",
    "'завод": "Стеклос
---------------------
Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского земельного отдела  
Расстрелян 1 июля 1938 года в Барнауле  


# Fine-tuning the gpt-4o-mini model
- It's not recommended to use fine-tuning to introduce new knowledge to the model. In this case, it's better to use RAG/Knowledge Grapths
- Fine-tuning helps with inheriting style, but not so much with hallucinations. The better solution to prevent the hallucinations for our case was RAG

We fine-tuned the mini model to see if there are any improvements on the examples where the mini model failed.

We used last year's (2023) human created cards as our ground truth for fine-tuning

In [9]:
# Read human generated cards for fine-tuning
url = "https://drive.google.com/file/d/1jAkv2B9EZcuqss8DftFUy2D4t-t6kdGd/view?usp=drive_link"
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df2023_cards = pd.read_csv(dwn_url)
print(f"Number of records: {len(df2023_cards)}")
duplicate_ids = df2023_cards.groupby('id').filter(lambda x: len(x) > 1)
print(f"Number of duplicate records: {len(duplicate_ids)}")
# dedupe data
df2023_cards = df2023_cards.drop_duplicates(subset=['id'])
print(f"NUmber of records after deduping: {len(df2023_cards)}")

# create a card from columns (the original card has a lot of spaces, so we will ignore it)
df2023_cards.loc[:,'card'] = df2023_cards.apply(lambda x: "\n".join([x['name'], x['age'], x['birth_info'], x['occupation'], x['death_info']]), axis=1)

# remove the word emphasis
df2023_cards.loc[:,'card'] = df2023_cards.card.map(lambda x: re.sub(chr(769),'',x))
df2023_cards.fillna('',inplace=True)

df2023_cards.head(2)

Number of records: 2972
Number of duplicate records: 146
NUmber of records after deduping: 2899


,filename,page_num,id,name,age,birth_info,occupation,death_info,card
0,2Imena_1258.pdf,0,24363485000,ЯКОВ ГУСТАВОВИЧ ВАЙВАР,72 ГОДА,РОДИЛСЯ В ЛИФЛЯНДСКОЙ ГУБЕРНИИ,"ПЕНСИОНЕР, ЖИЛ ВСМОЛЕНСКЕ",РАССТРЕЛЯН 11 ЯНВАРЯ 1938 ГОДА,ЯКОВ ГУСТАВОВИЧ ВАЙВАР\n72 ГОДА\nРОДИЛСЯ В ЛИФ...
1,2Imena_1258.pdf,1,219618741000,АНДРЕЙ КОНСТАНТИНОВИЧ КОНСТАНТИНОВ,55 ЛЕТ,РОДИЛСЯ И ЖИЛ ВТВЕРСКОЙ ОБЛАСТИ,ЧЕРНОРАБОЧИЙ В КОНТОРЕ ЗАГОТ-СЕНО,РАССТРЕЛЯН 29 ОКТЯБРЯ 1941,АНДРЕЙ КОНСТАНТИНОВИЧ КОНСТАНТИНОВ\n55 ЛЕТ\nРО...


In [10]:
# Get intersection of the human generated cards and ids in DB
# For some reason about 200 cards aren't presented in the db (could be that our slicing for this year was different than in 2023)
df2023_cards = df2023_cards[['id','card']].merge(df, on=['id'], how='inner')
print(f"Number of human generated cards that have input info: {len(df2023_cards)}")

Number of human generated cards that have input info: 857


In [18]:
# Pick good examples from human generated cards

def clean_cards(x):
  # Cleans data as some cards were read without spaces
  x = x.replace('ВЛЕНИНГРАДЕ', 'В ЛЕНИНГРАДЕ')
  x = x.replace('ВРОСТОВЕ-НА-ДОНУ', 'В РОСТОВЕ-НА-ДОНУ')
  x = x.replace('ВКАРЕЛИИ', 'В КАРЕЛИИ')
  x = x.replace('ВЕРЕВАНЕ', 'В ЕРЕВАНЕ')
  return x

df2023_cards.card = df2023_cards.card.map(clean_cards)

ids = [1295418, 3080332, 200025, 590352, 1700077, 396410, 848082, 1697207, 415360, 719271,
       922264, 977191,782888, 1087431, 530353, 780628, 1718445, 1065886, 975489, 935140,
       935140, 214138, 928699, 2999350, 773859, 1729253]
examples_human_generated = df2023_cards.loc[df2023_cards.id.isin(ids)]
examples_human_generated.loc[:, 'card'] = examples_human_generated.card.map(clean_cards)

In [19]:
# Create good examples from the known issues recorded in the returning_the_names github

ids_w_issues = [
        887131, # wrong expantion НАУРПа: https://github.com/compositor/returning_the_names/issues/31
        667429, # Russian grammar error of the М. Карай expantion (was "в селе Малая Карай", need "в селе Малай Карай"): https://github.com/compositor/returning_the_names/issues/30
        687694, # wrong expantion ТАССР (need all in Russian, but was Татарской Автономной Советской Socialistической Республики): https://github.com/compositor/returning_the_names/issues/24
        753186, # wrong expantion АКО -> (wronly) Артеклособанк -> (need) Акционерное Камчатское общество: https://github.com/compositor/returning_the_names/issues/31
        1787037, # wrong gramma was расстрелЕн, need расстрелян
        1222123, # wrong expantion of гор30: https://github.com/compositor/returning_the_names/issues/20
        1104124, # wrong expantion of КРАСНЫЙ КУТ?
        405685, # wrong expantion of SOCIALIST REPUBLIC, need Карельской АССР
]

In [33]:
# Generate the good outputs from the issues using the production model as we know it creates good cards

async def generate_cards(record, input_to_client=input_to_client, openai_client=client):

  record = RawCard(**record)
  messages = input_to_client.get_messages(record)

  completion = await openai_client.chat.completions.create(
            model=input_to_client.model, messages=messages, temperature=0
        )
  result = completion.choices[0].message.content

  return result

examples_with_issues = df.loc[df.id.isin(ids_w_issues), KEEP_COLS]

# generate outputs through production model
created_outputs = await asyncio.gather(*[generate_cards(x, input_to_client, client) for x in examples_with_issues.to_dict(orient='records')])

examples_with_issues['card'] = created_outputs

In [35]:
# Let's store our examples for the future reference
from pathlib import Path

# we are in the returning_the_names directory
Path("../data").mkdir(parents=True, exist_ok=True)

examples_with_issues.to_csv("../data/examples_with_issues.csv")

In [39]:
# Combine two sets of examples for fine-tuning
examples_all = pd.concat([examples_human_generated, examples_with_issues], ignore_index=True)

In [53]:
# Create jsonl data to upload as file to openai
def create_messages_openai(x, input_to_client=input_to_client):

  record = RawCard(**x[KEEP_COLS])
  messages = input_to_client.get_messages(record)

  # LLM ideal answer
  messages.append({"role": "assistant", "content": x['card']})

  return messages

# the column name should be 'messages'
examples_all['messages'] = examples_all.apply(lambda x: create_messages_openai(x), axis=1)

# jsonl file to upload to openai
examples_all[['messages']].to_json("../data/cards_good_examples.jsonl", orient='records', lines=True)

In [54]:
# Upload the training file ton openai
from openai import OpenAI

client_fine_tuning = OpenAI(api_key=OPENAI_KEY)
file_id = client_fine_tuning.files.create(
  file=open("../data/cards_good_examples.jsonl", "rb"),
  purpose='fine-tune'
)

In [55]:
# Fine-tune the model
client_fine_tuning.fine_tuning.jobs.create(
  training_file=file_id.id,
  model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-qGpWR3wvTaWP3sCIRdKil2fa', created_at=1733021561, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-zmirdOm1LZQNsHVYVocMI6jL', result_files=[], seed=1152445145, status='validating_files', trained_tokens=None, training_file='file-Y1bb13YVY5gm8qnVe5Jtjd', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [58]:
# Check the status of the job
# after the job is finished (filled finished_at), get the fine_tuned_model
client_fine_tuning.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-qGpWR3wvTaWP3sCIRdKil2fa', created_at=1733021561, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AZURfnFa', finished_at=1733021874, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-zmirdOm1LZQNsHVYVocMI6jL', result_files=['file-TV9Kj1RRzHsLLYeEFccV8L'], seed=1152445145, status='succeeded', trained_tokens=497493, training_file='file-Y1bb13YVY5gm8qnVe5Jtjd', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-WpOJq9DrQaCK2g3ZsMsZ4Oqn', created_at=1733021294, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Unexpected file format, expected either prompt/completion pairs or chat messages.', param='training_file'), fine_tuned_model=None, finished_at

In [44]:
fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AZURfnFa'

Check the examples with the issues. They are now fixed! Of course, the fine-tuned model won't fix the unseen issues. So be careful.

In [60]:
id = 1222123 # Зав. сектором горЗО -> Заведующий сектором городского земельного отдела

model = fine_tuned_model

record = df.loc[df.id == id, KEEP_COLS].to_dict(orient='records')[0]
record = RawCard(**record)
card = record.pretty_print_without_id()
print(card)
print(f"The prod model stems: {record.get_stems()}")
print()
custom = ai_prompts.get_sub(record.get_stems())
print(f"The length of the additional prompt created through RAG: {len(custom.split(chr(10)))}")
print("--------------------------------------------")

# no tunned model
messages = input_to_client.get_messages(record)
completion = await client.chat.completions.create(
            model="gpt-4o-mini", messages=messages, temperature=0
        )
result = completion.choices[0].message.content
print("No tunned model with the current RAG:\n")
print(result)
print("--------------------------------------------")
print()

# the fine-tunned model
messages = input_to_client.get_messages(record)
completion = await client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )
result = completion.choices[0].message.content
print("The fine-tuned model with the current RAG:\n")
print(result)
print()




Имя Отчество Фамилия: Венедикт Иванович Катков  
Возраст: 31
Пол: мужчина
Место рождения: None
Род занятий: Зав. сектором горЗО
Дата расстрела: 01.07.1938
Место расстрела: г. Барнаул      

The prod model stems: {'сектор', 'зав', 'барна', 'гор'}

The length of the additional prompt created through RAG: 61
--------------------------------------------
No tunned model with the current RAG:

Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского отдела  
Расстрелян 1 июля 1938 года в Барнауле  
--------------------------------------------

The fine-tuned model with the current RAG:

Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского земельного отдела в Барнауле  
Расстрелян 1 июля 1938 года в Барнауле  



In [61]:
id = 667429 # Малая Карай fixed also, now it's МАЛЫЙ КАРАЙ

model = fine_tuned_model

record = df.loc[df.id == id, KEEP_COLS].to_dict(orient='records')[0]
record = RawCard(**record)
card = record.pretty_print_without_id()
print(card)
print(f"The prod model stems: {record.get_stems()}")
print()
custom = ai_prompts.get_sub(record.get_stems())
print(f"The length of the additional prompt created through RAG: {len(custom.split(chr(10)))}")
print("--------------------------------------------")

# no tunned model
messages = input_to_client.get_messages(record)
completion = await client.chat.completions.create(
            model="gpt-4o-mini", messages=messages, temperature=0
        )
result = completion.choices[0].message.content
print("No tunned model with the current RAG:\n")
print(result)
print("--------------------------------------------")
print()

# the fine-tunned model
messages = input_to_client.get_messages(record)
completion = await client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )
result = completion.choices[0].message.content
print("The fine-tuned model with the current RAG:\n")
print(result)
print()


Имя Отчество Фамилия: Алексей Егорович Бесчетнов  
Возраст: 33
Пол: мужчина
Место рождения: Саратовская обл., Романовский р-н, с. М. Карай
Род занятий: крестьянин-единоличник.
Дата расстрела: 04.05.1930
Место расстрела: г. Балашов      

The prod model stems: {'балаш', 'саратовск', 'кар', 'крестьянинединоличн', 'романовск', 'обл', 'рн'}

The length of the additional prompt created through RAG: 58
--------------------------------------------
No tunned model with the current RAG:

Бесчетнов Алексей Егорович  
33 года  
Родился в селе Малая Карай Саратовской области  
Крестьянин-единоличник  
Расстрелян 4 мая 1930 года в Балашове  
--------------------------------------------

The fine-tuned model with the current RAG:

АЛЕКСЕЙ ЕГОРОВИЧ БЕСЧЕТНОВ
33 ГОДА
РОДИЛСЯ В СЕЛЕ МАЛЫЙ КАРАЙ РОМАНОВСКОГО РАЙОНА САРАТОВСКОЙ ОБЛАСТИ
КРЕСТЬЯНИН-ЕДИНОЛИЧНИК
РАССТРЕЛЯН 4 МАЯ 1930 В БАЛАШОВЕ



# RAG - Retrieval-Augmented Generation
The production model has RAG, but it's implemented in a straight-forward way:
  - uses a trie structure to store all abbreviations and their meanings
    - keys are normalized abbreviations concatinated with their normalized meaning (normalized means only letters are left)
    - values are abbreviations and their meanings
  - for each request creates a list of words from the birth place, occupation and place of execution.  
  - for each word extracts all values from the trie that starts with the word.


See the below the production model's implementation of RAG

In [16]:
import pygtrie
from src.ai_prompts import _FROM_SITE_ABBREVIATIONS, _FROM_BOOK_ABBREVIATIONS # these are copy-pasted abbreviations
from src.ai_prompts import _init, normalize

_abbreviations: pygtrie.CharTrie = pygtrie.CharTrie()
def _init() -> None:
    all_abbreviations = _FROM_SITE_ABBREVIATIONS + "\n" + _FROM_BOOK_ABBREVIATIONS
    for line in all_abbreviations.splitlines():
        _abbreviations[normalize(line)] = line
_init()

for key, value in _abbreviations.items()[:4]:
    print(f"Key is **{key}**, value is **{value}**")

Key is ****, value is ****
Key is **стстанция**, value is **ст - станция**
Key is **стстанциястарший**, value is **ст. — станция, старший**
Key is **стстстарыйстильстарогостиля**, value is **    "ст.": "ст. — старый стиль, старого стиля",**


For the word 'ст' the trie RAG extracts all values that start with 'ст'

In [27]:
# # For the word 'ст' the trie RAG extracts all values that start with 'ст'
_abbreviations.values('ст')

['ст - станция',
 'ст. — станция, старший',
 '    "ст.": "ст. — старый стиль, старого стиля",',
 '    "СТО": "Совет труда и обороны",',
 'стр-во — строительство',
 'строит. — строительный',
 '    "строй": "строительство, строительный, -ая, -ое",',
 '    "стройгаз": "Строительный Трест по расширению* Горьковского автомобильного завода им. Молотова",',
 '    "Стройгазхим": "Управление строительства Ленинградского газохимического комбината",',
 '    "стройдеталь": "строительная деталь",',
 '    "Стройдормаш": "Отдел строительного и дорожного машиностроения",',
 '    "Стройиздат": "Государственное издательство строительной литературы",',
 '    "стройном": "строительный комитет",',
 '    "Стройматериал": "Трест производства строительных материалов",',
 '    "Строймеханизация": "Государственная специальная контора по механизации строительных работ",',
 '    "Стромбюро": "Всесоюзное бюро промышленности строительных материалов",',
 '    "стромор": "морской строительный отдел",',
 '    "Стромсо

The production RAG has two problems:
- extracts a lot of information that is noise. We want better matches
- if there is no exact match, then there won't be empty RAG. We want to extract at least abbreviations close to meaning. For example, 'стр-во' is stored in the trie as 'стрво', but what if we see in the text 'стро-во'. Then the trie won't return anything as there is no node 'строво', but we want the RAG return the values from the node 'стр-во' as it has close meaning.

Hence we implemented the RAG through embeddings which should combat above shortcomings.

Before we continue, we did a few things outside of this notebook:
  - Exported the russian abbreviations from pdf file to csv
  - Created the embeddings for abbreviations (only for abbreviations and not for the meaning)
  - Stored abbreviations' embeddings, abbreviations and its meaning to a csv file.

The code below downloads this csv file into the notebook



In [11]:
# Download the abbreviations csv file.
# It is a large file, so we need to download it first before reading into pandas
import gdown

url = "https://drive.google.com/uc?id=1-hvSddpXcHmB9Mr3o3ORj5-XvDGqbuOt"
output = "data/ru_abbr_dictionary_embedding.csv"
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1-hvSddpXcHmB9Mr3o3ORj5-XvDGqbuOt
From (redirected): https://drive.google.com/uc?id=1-hvSddpXcHmB9Mr3o3ORj5-XvDGqbuOt&confirm=t&uuid=cb6b5fb0-ddef-4e0a-a84c-96bcbe8a8485
To: /content/drive/MyDrive/Projects/optimizing_returning_the_names/returning_the_names/data/ru_abbr_dictionary_embedding.csv
100%|██████████| 246M/246M [00:03<00:00, 78.5MB/s]


'data/ru_abbr_dictionary_embedding.csv'

In [29]:
# Read the abbreviations dictionary as pandas dataframe
import ast

abbreviations_df = pd.read_csv("data/ru_abbr_dictionary_embedding.csv")
abbreviations_df.drop(columns=['Unnamed: 0'], inplace=True)
abbreviations_df['embedding'] = abbreviations_df.embedding.map(lambda x: ast.literal_eval(x))
abbreviations_df.fillna('',inplace=True)
print(f"Before: {len(abbreviations_df)}")
abbreviations_df = abbreviations_df.loc[abbreviations_df.embedding.map(lambda x: len(x) > 0)]
print(f"After: {len(abbreviations_df)}")

Before: 7168
After: 7168


In [31]:
abbreviations_df.head(3)

,abbr,meaning,embedding
0,А,"армия, ангстрем, ампер","[0.009917707182466984, -0.005803981330245733, ..."
1,АА,Академия архитектуры,"[0.005386947188526392, -0.0027102031745016575,..."
2,ААН,Академия артиллерийских наук,"[-0.010861051268875599, 0.016613828018307686, ..."



The RAG through embeddings is implemented through the following steps:
  - Given request to create a card, get place of birth, occupation and execution place as a concatinated string. This is our query.
  - For each word in the query get its stem.
  - For each stem get its embedding.
  - For each stem's embedding find the close embeddings from the abbreviation dictionary (top 100 which closer than 0.5) and return the list of abbreviations and its meanings that have the closest embeddings
  - Compose a message to GPT out of these abbreviations and its meanings (just tell the GPT that it needs to lookup on these abbreviations' references before expanding abbreviations)

Disclaimer: the code below is taken from the openai tutorials and slightly modified for our needs

In [34]:
from typing import Dict, List
from nltk.stem.snowball import SnowballStemmer

# search function
async def strings_ranked_by_relatedness(
    client: AsyncOpenAI,
    query: str,
    reference_df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    model="text-embedding-3-small",
    top_n: int = 100,
    threshold=0.5
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = await client.embeddings.create(
        model=model,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (' : '.join([row["abbr"], row['meaning']]), relatedness_fn(query_embedding, row["embedding"]))
        for i, row in reference_df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings_and_relatednesses = [(string, rel) for string, rel in strings_and_relatednesses if rel > threshold]
    if strings_and_relatednesses:
      strings, relatednesses = zip(*strings_and_relatednesses)
      return strings[:top_n], relatednesses[:top_n]
    else:
      return [], []

def get_stems(words: str) -> List[str]:

  stemmer = SnowballStemmer("russian")

  stemmed_words = [stemmer.stem(word) for word in words.split()]

  return stemmed_words

async def get_reference(
    client: AsyncOpenAI,
    query: str,
    reference_df: pd.DataFrame,
    model: str,
    top_n: int = 10,
    threshold: float = 0.5,
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""

    stems = get_stems(query)
    ref_dic = ''
    for stem in stems:
      strings, _ = await strings_ranked_by_relatedness(client, stem, reference_df, model=model, top_n=top_n, threshold=threshold)
      ref_dic += '\n'.join(strings) + '\n'
    return ref_dic

Let's see how the functions work. Below an example of "НАУРПа" the list of abbreviations and its meaning returned that are close to "НАУРПа":
- The correct abbreviation is in the top of the returned list which is good as the mini model looks only on the top for relevant abbreviations. We assume the full model do the same, only the top for the full model is a bigger list.
- The returned list is short and has all relevant information which helps with accuracy and cost

In [ ]:
strings, relatednesses = await strings_ranked_by_relatedness(client, "НАУРПа", abbreviations_df, model="text-embedding-3-small", top_n=200)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.938


'НАУРП : Нижне-Амурское управление речных пароходств'

relatedness=0.610


'НУАРМ : Политическое управление армии'

relatedness=0.585


...

'УР : уравнительный рубеж'

relatedness=0.502


'КНАП : Комбинат наглядной агитации и пропаганды'

relatedness=0.501


'НАД : начальник авиационной дивизии, начальник артиллерии дивизии'

Not everything is rosy for "горЗО" example as the relevant abbreviation is 5th in the list. The mini model doesn't like that and returns the first abbreviation instead "гор- : горный, -ая, -ое, городской, -ая, -ое"

In [ ]:
# examples: Зав. сектором горЗО
strings, relatednesses = await strings_ranked_by_relatedness(client, "горЗО", abbreviations_df, model="text-embedding-3-small", threshold=0.5, top_n=200)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.722


'гор- : горный, -ая, -ое, городской, -ая, -ое'

relatedness=0.705


'гор. : город, городок'

relatedness=0.704


...

'горком : городской комитет'

relatedness=0.502


'Гипрогор : Государственный Институт по проектированию городов, гражданских сооружений, по съемкам и по планированию населенных мест'

relatedness=0.501


'зппо : заградительный противотанковый переносный огонь'

It became better when we used stems instead of words for matching. Below the "горЗО" example improved.

In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("russian")

words = "Зав. сектором горЗО".split()
stemmed_words = [stemmer.stem(word) for word in words]

print("\nPorter Stemmer Output:")
for original, stemmed in zip(words, stemmed_words):
    print(f"{original} -> {stemmed}")


Porter Stemmer Output:
Зав. -> зав.
сектором -> сектор
горЗО -> горз


For stem "горз" the relevant result is on the top of the list and the list is much shorter.

it might be we wouldn't need to use stems if we had a better embedding model. We used the simplest and cheapest one.

In [ ]:
# example: Зав. сектором горЗО
strings, relatednesses = await strings_ranked_by_relatedness(client, "горз", abbreviations_df, model="text-embedding-3-small", threshold=0.5, top_n=200)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.886


'горзо : городской земельный отдел'

relatedness=0.771


'горздрав : городской отдел здравоохранения'

relatedness=0.686


...

'горОНО (гороно) : городской отдел народного образования'

relatedness=0.521


'помзаворг : помощник заведующего организацией'

relatedness=0.502


'горпром : горная промышленность'

Now the final code to generate a card from a request through RAG embeddings

In [39]:
async def ask(
    client: AsyncOpenAI,
    record: RawCard,
    card: str,
    abbreviations_df: pd.DataFrame = abbreviations_df,
    model: str = input_to_client.model,
    embedding_model: str = 'text-embedding-3-small',
    top_n: int = 10,
    threshold: float = 0.5,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    query = [record.place_of_birth, record.occupation, record.execution_place]
    query = ' '.join([x for x in query if x])
    # query = record.occupation
    reference_dic = await get_reference(client, query, abbreviations_df, model=embedding_model, top_n=top_n, threshold=threshold)
    reference_dic = '\nИспользуй этот справочник для расшифровки аббревиатур:\n' + reference_dic
    messages = [
        {"role": "system", "content": ai_prompts.ALL_IN_ONE_PROMPT},
        {"role": "system", "content": reference_dic},
        {"role": "assistant","content": ai_prompts.GOOD_EXAMPLES},
        {"role": "user", "content": card},
    ]
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message, messages

In [40]:
id = 1222123 # example: Зав. сектором горЗО
record = df.loc[df.id == id, KEEP_COLS].to_dict(orient='records')[0]
record = RawCard(**record)
card = record.pretty_print_without_id()
print(card)
print()

clean_card, messages = await ask(client, record, card, model='gpt-4o-mini',top_n=200, threshold=0.5, print_message=True)
print(clean_card)


Имя Отчество Фамилия: Венедикт Иванович Катков  
Возраст: 31
Пол: мужчина
Место рождения: None
Род занятий: Зав. сектором горЗО
Дата расстрела: 01.07.1938
Место расстрела: г. Барнаул      


Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского земельного отдела  
Расстрелян 1 июля 1938 года в Барнауле  


In [41]:
# See the generated reference dictionary through RAG embeddings
print(messages[1]['content'])


Используй этот справочник для расшифровки аббревиатур:
зав. : завод (карт.)
зав- : заведующий, -ая, заводской, -ая, -ое
завуч : заведующий учебной частью
завмаг : заведующий магазином
ЗавВО : Заволжский военный округ
завхим : заведующий химической службой
заветча : заведующий ветеринарной частью
ав. : авиация, авиационный, -ая -ое
коопзав : заведующий кооперативом
замзав : заместитель заведующего
завтео : заведующий техническим отделом
завобовосу : заведующий общим отделом- Окружного военносанитарного управления
завбиб : заведующий библиотекой
завканц : заведующий канцелярией
предзавкома : председатель заводского комитета
ЗВ : зажигательное вещество
завкомовец : член заводского комитета
помзав : помощник заведующего
вив : Всесоюзный трест искусственного волокна
вив : величина изменения высоты
зам. : замыкатель, заместитель
ЗВО : Заволжский военный округ
помзаворг : помощник заведующего организацией
завинт : заведующий интендантской частью
рзв. : разведывательный зенитный взвод
сект. :

# Fine-tuned mini-model with RAG with embeddings

Now let's have all together:
- Use RAG embedding to create a reference for expanding abbreviations
- Use fine-tuned model instead of pure openai models (we don't use good examples in the promps as we fine-tuned the model already)

In [42]:
async def ask2(
    client: AsyncOpenAI,
    record: RawCard,
    card: str,
    abbreviations_df: pd.DataFrame = abbreviations_df,
    model: str = input_to_client.model,
    embedding_model: str = 'text-embedding-3-small',
    top_n: int = 10,
    threshold: float = 0.5,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    # query = [record.occupation,record.place_of_residence, record.execution_place, record.place_of_birth]
    query = record.occupation
    reference_dic = await get_reference(client, query, abbreviations_df, model=embedding_model, top_n=top_n, threshold=threshold)
    reference_dic = '\nИспользуй этот справочник для расшифровки аббревиатур:\n' + reference_dic
    messages = [
        {"role": "system", "content": ai_prompts.ALL_IN_ONE_PROMPT},
        {"role": "system", "content": reference_dic},
        # No need to add good examples to the prompt anymore
        # {"role": "assistant","content": ai_prompts.GOOD_EXAMPLES},
        {"role": "user", "content": card},
    ]
    response = await client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response.choices[0].message.content
    return response_message, messages

In [45]:
id = 1222123
record = df.loc[df.id == id, KEEP_COLS].to_dict(orient='records')[0]
record = RawCard(**record)
card = record.pretty_print_without_id()
print(card)
print()

clean_card, messages = await ask2(client, record, card, model=fine_tuned_model,top_n=200, threshold=0.5, print_message=True)
print(clean_card)


Имя Отчество Фамилия: Венедикт Иванович Катков  
Возраст: 31
Пол: мужчина
Место рождения: None
Род занятий: Зав. сектором горЗО
Дата расстрела: 01.07.1938
Место расстрела: г. Барнаул      


Венедикт Иванович Катков  
31 год  
Место рождения неизвестно  
Заведующий сектором городского земельного отдела в Барнауле  
Расстрелян 1 июля 1938 года в Барнауле  


# Conclusion

We were able to improve the gpt mini model through RAG with embeddings and fine-tuning. Now the mini model doesn't hallucinates and creates card with the stylistic we needed.

There are some things can be done further:
- Combine the production RAG through trie with RAG through embeddings, but make the final list smaller. Just in case to make sure we are coverying everything we need